<a href="https://colab.research.google.com/github/mirasmitty/Project_Zillow/blob/main/zillow_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructor Do: Random Forests

In [ ]:
#USING SPARK VIA GOOGLE COLLAB CODE TO MAKE IT WORK
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Initial imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("ZillowData").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/mirasmitty/Project_Zillow/main/Resources/Zillow_data_Detroit.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("Zillow_data_Detroit.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+---------------+----------------------------+-------------------------------+
|Week of pending|Mean days listing to pending|Mean price reduction percentage|
+---------------+----------------------------+-------------------------------+
|      1/27/2018|                          49|                    0.045619162|
|       2/3/2018|                          48|                    0.046532464|
|      2/10/2018|                          47|                    0.048535741|
|      2/17/2018|                          46|                    0.049168844|
|      2/24/2018|                          43|                    0.048167891|
|       3/3/2018|                          42|                    0.047288417|
|      3/10/2018|                          40|                     0.04445831|
|      3/17/2018|                          38|                    0.042958179|
|      3/24/2018|                          37|                    0.042379745|
|      3/31/2018|                          35|      

In [ ]:
# Print our schema
df.printSchema()

root
 |-- Week of pending: string (nullable = true)
 |-- Mean days listing to pending: string (nullable = true)
 |-- Mean price reduction percentage: string (nullable = true)



## Loading and Preprocessing Loans Encoded Data

In [ ]:
df.dtypes

[('Week of pending', 'string'),
 ('Mean days listing to pending', 'string'),
 ('Mean price reduction percentage', 'string')]

In [ ]:
df.count()

315

In [ ]:
# Define features set
x = df.drop("Mean price reduction percentage")

x.show()

+---------------+----------------------------+
|Week of pending|Mean days listing to pending|
+---------------+----------------------------+
|      1/27/2018|                          49|
|       2/3/2018|                          48|
|      2/10/2018|                          47|
|      2/17/2018|                          46|
|      2/24/2018|                          43|
|       3/3/2018|                          42|
|      3/10/2018|                          40|
|      3/17/2018|                          38|
|      3/24/2018|                          37|
|      3/31/2018|                          35|
|       4/7/2018|                          32|
|      4/14/2018|                          30|
|      4/21/2018|                          27|
|      4/28/2018|                          26|
|       5/5/2018|                          24|
|      5/12/2018|                          24|
|      5/19/2018|                          23|
|      5/26/2018|                          23|
|       6/2/2

In [ ]:
# Define target vector
y = df.select("Mean price reduction percentage").rdd.flatMap(lambda x: x).collect()

#y = df["Mean price reduction percentage"].ravel()

y[:5]

['0.045619162', '0.046532464', '0.048535741', '0.049168844', '0.048167891']

In [48]:
# Splitting into Train and Test sets
# Splitting into Train and Test sets train_data,

train_data, test_data = df.randomSplit([0.8, 0.2], seed=78)

x_train = train_data.select("Week of pending", "Mean days listing to pending")
y_train = train_data.select("Mean price reduction percentage")
x_test = test_data.select("Week of pending", "Mean days listing to pending")
y_test = test_data.select("Mean price reduction percentage")



In [49]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [54]:
# Fitting Standard Scaler
#x_scaler = scaler.fit(x_train)

# Initialize the StandardScaler
scaler = StandardScaler(inputCol=("Week of pending", "Mean days listing to pending"), outputCol="scaled_features", withMean=True, withStd=True)

TypeError: StandardScaler.__init__() got an unexpected keyword argument 'inputCol'

In [ ]:
# Scaling data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Fitting the Random Forest Model

In [ ]:
# Create a random forest classifier
rf_model = RandomForestRegressor(n_estimators=500, random_state=78)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(x_train_scaled, y_train)

## Making Predictions Using the Random Forest Model

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(x_test_scaled)

In [ ]:
predictions.shape

## Model Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_error


In [ ]:
# Compute the metrics for the Random forest regression model
score = rf_model.score(x_test, y_test,sample_weight=None)
r2 = r2_score(y_test,predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
std = np.std(y_test)
error = mean_absolute_error(y_test, predictions)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")
print(f"The error is {error} ")

## Feature Importance

In [ ]:
# Random Forests in sklearn will automatically calculate feature importance
imprtances = rf_model.feature_importances_
# We can sort the features by their importance
sorted(zip(rf_model.feature_importances_, x.columns), reverse= True)

In [ ]:
# Visualize the features by importance
importances_df = pd.DataFrame(sorted(zip(rf_model.feature_importances_, x.columns), reverse = True))
importances_df.set_index(importances_df[1], inplace = True)
importances_df.drop(columns = 1, inplace = True)
importances_df.rename(columns = {0: 'Feature Importances'}, inplace = True)
importances_sorted = importances_df.sort_values(by = 'Feature Importances')
importances_sorted.plot(kind = 'barh', color = 'lightgreen', title = 'Features Importances', legend = False)

In [ ]:
print("Shape")
print("Training data : ", df.shape)
print("Test data     : ", df.shape)

In [ ]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

s = (df.dtypes == 'int')
num_cols = list(s[s].index)
print("Integer variables:")
print(num_cols)

s = (df.dtypes == 'float')
num_cols = list(s[s].index)
print("Real variables:")
print(num_cols)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)

In [ ]:
plt.figure(figsize=(18,6))
plt.title('Heatmap of missing values')
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [ ]:
unique_values = []
for col in object_cols:
  unique_values.append(df[col].unique().size)
plt.figure(figsize=(18,6))
plt.title('No. Unique values of Categorical Features')
plt.xticks(rotation=90)
sns.barplot(x=object_cols,y=unique_values)

In [ ]:
plt.figure(figsize=(18,36))
plt.title('Categorical Features: Distribution')
plt.xticks(rotation=90)
index = 1
for col in object_cols:
  y = df_combined[col].value_counts()
  plt.subplot(11,4,index)
  plt.xticks(rotation=90)
  sns.barplot(x=list(y.index), y=y)
  index +=1

### **Fill up missing values:**
* Drop the features 'Alley', 'Fence', and 'MiscFeature'.

* Drop 'Utilities' feature, as all but one have the value 'AllPub'

* All entries with missing 'FirePlaceQu' have 'Fireplaces' = 0. Hence fill missing values with 'NA'.

* All but one entries with missing 'PoolQC' value have 'PoolArea' = 0. Use mode for missing value with non-zero PoolArea. Use 'NA' for the rest of the entries.

* **Basement features:** Fill missing values with 'NA' or '0'.

* **Garage features:** Fill missing values with 'NA' or '0'.

* **Remaining Integer and Real features:** fill up missing values with mean of the corresponding feature.

* **Remaining Categorical features:** fill up missing values with mode of the corresponding feature.

In [ ]:
#df_combined.drop(columns='Id',inplace=True); print('Drop Id \n')
#df_combined['MSZoning'] = df_combined['MSZoning'].fillna(df_combined['MSZoning'].mode()[0])
#df_combined['LotFrontage'] = df_combined['LotFrontage'].fillna(df_combined['LotFrontage'].mean())
#df_combined.drop(columns='Alley',inplace=True); print('Drop Alley \n')

#df_combined['Utilities'] = df_combined['Utilities'].fillna(df_combined['Utilities'].mode()[0])
#print(df_combined['Utilities'].value_counts())
#df_combined.drop(columns='Utilities',inplace=True); print('Drop Utilities \n')

#df_combined['Exterior1st'] = df_combined['Exterior1st'].fillna(df_combined['Exterior1st'].mode()[0])
#df_combined['Exterior2nd'] = df_combined['Exterior2nd'].fillna(df_combined['Exterior2nd'].mode()[0])
#df_combined['MasVnrType'] = df_combined['MasVnrType'].fillna(df_combined['MasVnrType'].mode()[0])
#df_combined['MasVnrArea'] = df_combined['MasVnrArea'].fillna(df_combined['MasVnrArea'].mean())
#df_combined['Electrical'] = df_combined['Electrical'].fillna(df_combined['Electrical'].mode()[0])
#df_combined['KitchenQual'] = df_combined['KitchenQual'].fillna(df_combined['KitchenQual'].mode()[0])
#df_combined['Functional'] = df_combined['Functional'].fillna(df_combined['Functional'].mode()[0])

#df_combined.loc[(df_combined['Fireplaces'] != 0) & (df_combined['FireplaceQu'].isnull()) ][['FireplaceQu','Fireplaces']]
#df_combined['FireplaceQu'] = df_combined['FireplaceQu'].fillna('NA'); print('FirePlaceQu: Fill NA values for missing values \n')

#df_combined.loc[(df_combined['PoolQC'].isnull()) & df_combined['PoolArea']>0][['PoolQC','PoolArea']]
#df_combined.at[2599,'PoolQC'] = df_combined['PoolQC'].mode()[0]; print('PoolQC: Use mode for missing value with non-zero PoolAre \n')
#df_combined['PoolQC'] = df_combined['PoolQC'].fillna('NA'); print('PoolQC: Use NA for remaining missing values \n')

#df_combined['SaleType'].fillna(df_combined['SaleType'].mode()[0],inplace=True)
#df_combined.drop(columns=['Fence','MiscFeature','SalePrice'],inplace=True); print('Drop Fence, MiscFeature and SalePrice\n')

# Basement Features
#df_combined.loc[df_combined['BsmtQual'].isnull()][['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']].head()
#df_combined.loc[df_combined['TotalBsmtSF'].isnull()][['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinSF1','BsmtFinType2','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','BsmtFullBath','BsmtHalfBath']]
#print('Fill missing values of Basement features with NA or 0 \n')
#df_combined['BsmtQual'] = df_combined['BsmtQual'].fillna('NA')
#df_combined['BsmtCond'] = df_combined['BsmtCond'].fillna('NA')
#df_combined['BsmtExposure'] = df_combined['BsmtExposure'].fillna('NA')
#df_combined['BsmtFinType1'] = df_combined['BsmtFinType1'].fillna('NA')
#df_combined['BsmtFinType2'] = df_combined['BsmtFinType2'].fillna('NA')

#df_combined['BsmtFinSF1'] = df_combined['BsmtFinSF1'].fillna(int(0))
#df_combined['BsmtFinSF2'] = df_combined['BsmtFinSF2'].fillna(int(0))
#df_combined['BsmtUnfSF'] = df_combined['BsmtUnfSF'].fillna(int(0))
#df_combined['TotalBsmtSF'] = df_combined['TotalBsmtSF'].fillna(int(0))
#df_combined['BsmtFullBath'] = df_combined['BsmtFullBath'].fillna(int(0))
#df_combined['BsmtHalfBath'] = df_combined['BsmtHalfBath'].fillna(int(0))

# Garage Features
# df_combined.loc[df_combined['GarageCond'].isnull()][['GarageType','GarageYrBlt','GarageFinish','GarageCars','GarageArea','GarageQual','GarageCond']].head()
#print('Fill missing values of Garage features with NA or 0 \n')
#df_combined['GarageType'] = df_combined['GarageType'].fillna('NA')
#df_combined['GarageFinish'] = df_combined['GarageFinish'].fillna('NA')
#df_combined['GarageCond'] = df_combined['GarageCond'].fillna('NA')
#df_combined['GarageQual'] = df_combined['GarageQual'].fillna('NA')
#df_combined['GarageCars'] = df_combined['GarageCars'].fillna(int(0))
#df_combined['GarageArea'] = df_combined['GarageArea'].fillna(int(0))
#df_combined['GarageYrBlt'] = df_combined['GarageYrBlt'].fillna(int(0))

In [ ]:
df.head()

##Check that all missing values have been taken care of.


In [ ]:
print(df.isnull().sum().sum())


#Data Preprocessing

###Import Onehot encoder to encode categorical features


In [ ]:
from sklearn.preprocessing import OneHotEncoder


In [ ]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ',len(object_cols))

In [ ]:

cat = df.dtypes[df.dtypes == "object"].index.tolist()

df[cat].nunique()

In [ ]:
enc = OneHotEncoder(sparse=False)
# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(cat)
encode_df.head()

In [ ]:
OH_encoder = OneHotEncoder(sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(df[object_cols]))
OH_cols.index = df.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = df.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

In [ ]:
df_final.head()

In [ ]:
# Check that the shapes are consistent

print('df_final shape:', df_final.shape)
print('df_train shape:', df.shape)
print('df_test shape:',  df.shape)

X_Train = pd.DataFrame(df_final[:1460])
X_Test  = pd.DataFrame(df_final[1460:])
Y_Train = df['Mean price reduction percentage']

print('\nCheck that the datasets are consistent:\n')
print('X_train shape', X_Train.shape)
print('Y_train shape:', Y_Train.shape)
print('X_test shape:',  X_Test.shape)

#Model selection and prediction


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Split the training set into training and validation set

X_train, X_valid, Y_train, Y_valid = train_test_split(X_Train, Y_Train, train_size=0.8, test_size=0.2,random_state=0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

model_RFR = RandomForestRegressor()
model_RFR.fit(X_train, Y_train)
Y_pred = model_RFR.predict(X_valid)
print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
# Compute the metrics for the Random forest regression model
score = model_RFR.score(X_valid, Y_valid, sample_weight=None)
r2 = r2_score(Y_valid,Y_pred)
mse = mean_squared_error(Y_valid, Y_pred)
rmse = np.sqrt(mse)
std = np.std(Y_valid)
error = mean_absolute_error(Y_valid, Y_pred)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")
print(f"The error is {error} ")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

model_GBR = GradientBoostingRegressor()
model_GBR.fit(X_train, Y_train)
Y_pred = model_GBR.predict(X_valid)
print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
# Compute the metrics for the Random forest regression model
score = model_GBR.score(X_valid, Y_valid, sample_weight=None)
r2 = r2_score(Y_valid,Y_pred)
mse = mean_squared_error(Y_valid, Y_pred)
rmse = np.sqrt(mse)
std = np.std(Y_valid)
error = mean_absolute_error(Y_valid, Y_pred)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")
print(f"The error is {error} ")

In [ ]:
from sklearn.linear_model import SGDRegressor

model_SGD = SGDRegressor()
model_SGD.fit(X_train, Y_train)
Y_pred = model_SGD.predict(X_valid)
print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
# Compute the metrics for the Random forest regression model
score = model_SGD.score(X_valid, Y_valid, sample_weight=None)
r2 = r2_score(Y_valid,Y_pred)
mse = mean_squared_error(Y_valid, Y_pred)
rmse = np.sqrt(mse)
std = np.std(Y_valid)
error = mean_absolute_error(Y_valid, Y_pred)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")
print(f"The error is {error} ")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBRegressor

model_XGBR = XGBRegressor(learning_rate=0.03,n_estimators=200,objective='reg:squarederror')
model_XGBR.fit(X_train,Y_train)
Y_pred = model_XGBR.predict(X_valid)
print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
plt.figure()
plt.title('Comparison of Sale Price of Predicted and Actual values')
plt.scatter(Y_Train,model_RFR.predict(X_Train),label='Random Forest')
plt.scatter(Y_Train,model_XGBR.predict(X_Train),label='XGB')
plt.legend()

In [ ]:
from sklearn.model_selection import GridSearchCV
model = XGBRegressor()

n_estimators   = [100, 200, 500]
learning_rates = [0.03,0.1,0.3]
objectives     = ['reg:squarederror']

# Define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators' : n_estimators,
    'learning_rate':learning_rates,
    'objective' : objectives
    }

grid_cv = GridSearchCV(estimator = model,
            param_grid = hyperparameter_grid,
            scoring = 'neg_mean_absolute_error',
            return_train_score = True)

grid_cv.fit(X_Train,Y_Train)

In [ ]:
grid_cv.best_score_


In [ ]:
grid_cv.best_estimator_


In [ ]:
#Y_pred = random_cv.predict(X_valid)
#print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
regressor = grid_cv.best_estimator_
Y_pred = regressor.predict(X_valid)
print(mean_absolute_error(Y_valid, Y_pred))

In [ ]:
plt.figure()
plt.title('Comparison of Price Reduction')
plt.scatter(Y_Train,model_RFR.predict(X_Train),label='Random Forest')
plt.scatter(Y_Train,model_XGBR.predict(X_Train),label='XGB')
plt.scatter(Y_Train,regressor.predict(X_Train),label='Best model')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Value')
plt.legend()


#Preparation of Submission Data

In [ ]:
#Y_Pred = regressor.predict(X_Test)


In [ ]:
#Y_Pred


In [ ]:
#Y_Pred.shape


In [ ]:
#sub = pd.DataFrame()
#sub['Mean days listing to pending'] = df['Mean days listing to pending']
#sub['Mean price reduction percentage'] = Y_Pred